In [1]:
from torch import nn
import torch

/opt/pytorch/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
import os, sys, json
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)

In [3]:
modules = nn.ModuleList([None])

In [11]:
from models.deepseek.model import ExpertMLA, ModelArgs, ExpertType

In [4]:
from models.deepseek import model

In [6]:
os.path.join(parent, 'models/deepseek')

'/home/ubuntu/projects/llm-models/llm-models/models/deepseek'

In [17]:
import importlib
importlib.reload(model)
torch.set_default_dtype(torch.bfloat16)

In [6]:
args_expert = model.ModelArgs(expert_type=model.ExpertType(2))

In [14]:
args_expert = model.ModelArgs(expert_type=model.ExpertType(2),
                             dim=512, inter_dim=1365, moe_inter_dim=256,
                             kv_lora_rank=128, qk_nope_head_dim=32, qk_rope_head_dim=16, 
                             v_head_dim=32, 
                             n_routed_experts=4, n_layers=4)

In [22]:
expert_mla = model.ExpertMLA(args_expert)

In [15]:
expert_model = model.Transformer(args_expert)

step1
step4
step2
step3
step1
step4
step1
step2
step3
step1
step2
step3
step1
step2
step3
step1
step2
step3
step2
step3
step1
step4
step1
step2
step3
step1
step2
step3
step1
step2
step3
step1
step2
step3
step2
step3
step1
step4
step1
step2
step3
step1
step2
step3
step1
step2
step3
step1
step2
step3
step2
step3


In [8]:
expert_mla

ExpertMLA(
  (wq): Linear()
  (wkv_a): Linear()
  (kv_norm): RMSNorm()
  (wkv_b): Linear()
  (wo): Linear()
)

In [8]:
test_batch_size = 16
test_seq_size = 12
test_input = torch.zeros((test_batch_size*test_seq_size, 1, args_expert.dim), dtype=torch.bfloat16)

In [9]:
test_input.shape

torch.Size([192, 1, 2048])

In [7]:
freq_cis = model.precompute_freqs_cis(args_expert)

In [25]:
output = expert_mla(test_input, freq_cis[:1,:], None)

torch.Size([192, 1, 16, 4])
torch.Size([4, 16, 128])


In [13]:
import math

In [10]:
block = model.Block(-1, args_expert, True)

step1
step2
step3


In [13]:
output = block(test_input, 0, freq_cis[:1,:], None)

In [22]:
output.shape

torch.Size([192, 1, 2048])

In [20]:
expert = model.Expert(args_expert)

In [21]:
output = expert(test_input, 0, freq_cis[:1,:], None)

In [13]:
moe_attention = model.MoE(args_expert)

step1
step2
step3
step1
step2
step3
step1
step2
step3
step1
step2
step3


In [31]:
moe.get_submodule('experts')[0].get_submodule('block').get_submodule('attn').get_submodule('wq').weight.shape

torch.Size([3072, 2048])

In [46]:
for k, v in moe_attention.named_parameters():
    print(k)
    print(v.shape)

gate.weight
torch.Size([64, 2048])
experts.0.block.attn.learnable_k_cache
torch.Size([4, 16, 192])
experts.0.block.attn.learnable_v_cache
torch.Size([4, 16, 128])
experts.0.block.attn.wq.weight
torch.Size([3072, 2048])
experts.0.block.attn.wkv_a.weight
torch.Size([576, 2048])
experts.0.block.attn.kv_norm.weight
torch.Size([512])
experts.0.block.attn.wkv_b.weight
torch.Size([4096, 512])
experts.0.block.attn.wo.weight
torch.Size([2048, 2048])
experts.0.block.attn_norm.weight
torch.Size([2048])
experts.1.block.attn.learnable_k_cache
torch.Size([4, 16, 192])
experts.1.block.attn.learnable_v_cache
torch.Size([4, 16, 128])
experts.1.block.attn.wq.weight
torch.Size([3072, 2048])
experts.1.block.attn.wkv_a.weight
torch.Size([576, 2048])
experts.1.block.attn.kv_norm.weight
torch.Size([512])
experts.1.block.attn.wkv_b.weight
torch.Size([4096, 512])
experts.1.block.attn.wo.weight
torch.Size([2048, 2048])
experts.1.block.attn_norm.weight
torch.Size([2048])
experts.2.block.attn.learnable_k_cache
to

In [20]:
args_regular = model.ModelArgs(expert_type=model.ExpertType(1),
                             dim=512, inter_dim=1365, moe_inter_dim=256,
                             kv_lora_rank=128, qk_nope_head_dim=32, qk_rope_head_dim=16, 
                             v_head_dim=32)
moe_regular = model.MoE(args_regular)

In [11]:
moe_attention

MoE(
  (gate): Gate()
  (experts): ModuleList(
    (0-63): 64 x Expert(
      (block): Block(
        (attn): ExpertMLA(
          (wq): Linear()
          (wkv_a): Linear()
          (kv_norm): RMSNorm()
          (wkv_b): Linear()
          (wo): Linear()
        )
        (attn_norm): RMSNorm()
      )
    )
  )
  (shared_experts): MLP(
    (w1): ColumnParallelLinear()
    (w2): RowParallelLinear()
    (w3): ColumnParallelLinear()
  )
)

In [9]:
def compute_model_memory(model: nn.Module) -> tuple[int, str]:
    """
    Computes the total memory size of a torch.nn.Module (parameters and buffers).

    Args:
        model: The PyTorch nn.Module instance.

    Returns:
        A tuple containing (total_bytes, formatted_size_string).
    """
    # 1. Calculate memory for parameters (weights, biases)
    param_bytes = sum(p.nelement() * p.element_size() for k, p in model.named_parameters())

    # 2. Calculate memory for buffers (running_mean, running_var, etc.)
    buffer_bytes = sum(b.nelement() * b.element_size() for b in model.buffers())

    total_bytes = param_bytes + buffer_bytes

    # Format for readability
    if total_bytes > (1024**3):
        formatted_size = f"{total_bytes / (1024**3):.2f} GB"
    elif total_bytes > (1024**2):
        formatted_size = f"{total_bytes / (1024**2):.2f} MB"
    elif total_bytes > 1024:
        formatted_size = f"{total_bytes / 1024:.2f} KB"
    else:
        formatted_size = f"{total_bytes} bytes"

    return total_bytes, formatted_size

In [21]:
compute_model_memory(moe_regular)

(270008320, '257.50 MB')

In [16]:
compute_model_memory(expert_model)


(1666917376, '1.55 GB')

In [5]:
572784640/1e9*27


15.465185279999998